In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame()
df = pd.read_csv('../input/russian-authors/russian_authors_12.csv', encoding='utf-8')
df.head(3)

,sentence,author
0,А живет он второе лето в этом вонючем городиш...,Chehov
1,Вдруг она услыхала ровный и спокойный носовой...,Tolstoy
2,"""Послушайте, матушка.",Gogol


In [4]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

review_lines = list()
lines = df['sentence'].values.tolist()

for line in lines:   
    tokens = word_tokenize(line)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word    
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words    
    stop_words = set(stopwords.words('russian'))
    words = [w for w in words if not w in stop_words]
    review_lines.append(words)

In [5]:
len(review_lines)

22500

In [18]:
import gensim 

EMBEDDING_DIM = 300
# train word2vec model
model = gensim.models.Word2Vec(sentences=review_lines, size=EMBEDDING_DIM, window=1, workers=4, min_count=1)
# vocab size
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

Vocabulary size: 47967


In [19]:
# save model in ASCII (word2vec) format
filename = 'russian_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

In [20]:
import os

embeddings_index = {}
f = open(os.path.join('', 'russian_embedding_word2vec.txt'),  encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

In [21]:
total_reviews = df['sentence'].values
max_length = max([len(s.split()) for s in total_reviews])

In [22]:
print(max_length)

199


In [23]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

VALIDATION_SPLIT = 0.2

# vectorize the text samples into a 2D integer tensor
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(review_lines)
sequences = tokenizer_obj.texts_to_sequences(review_lines)

# pad sequences
word_index = tokenizer_obj.word_index
print('Found %s unique tokens.' % len(word_index))

review_pad = pad_sequences(sequences, maxlen=max_length)
labelencoder_X = LabelEncoder()
author = df['author'].values
author = pd.DataFrame(labelencoder_X.fit_transform(df['author']))
onehotencoder = OneHotEncoder(categorical_features = [0])
author = pd.DataFrame(onehotencoder.fit_transform(author).toarray())
author = np.array(author)
author = author.astype(dtype = 'int32')

print('Shape of review tensor:', review_pad.shape)
print('Shape of sentiment tensor:', author.shape)



Found 47967 unique tokens.
Shape of review tensor: (22500, 199)
Shape of sentiment tensor: (22500, 5)


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [24]:
# split the data into a training set and a validation set
indices = np.arange(review_pad.shape[0])
np.random.shuffle(indices)
review_pad = review_pad[indices]
author = author[indices]
num_validation_samples = int(VALIDATION_SPLIT * review_pad.shape[0])

X_train_pad = review_pad[:-num_validation_samples]
y_train = author[:-num_validation_samples]
X_test_pad = review_pad[-num_validation_samples:]
y_test = author[-num_validation_samples:]

In [25]:
print('Shape of X_train_pad tensor:', X_train_pad.shape)
print('Shape of y_train tensor:', y_train.shape)

print('Shape of X_test_pad tensor:', X_test_pad.shape)
print('Shape of y_test tensor:', y_test.shape)

Shape of X_train_pad tensor: (18000, 199)
Shape of y_train tensor: (18000, 5)
Shape of X_test_pad tensor: (4500, 199)
Shape of y_test tensor: (4500, 5)


In [26]:
EMBEDDING_DIM = 300
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [27]:
print(num_words)

47968


In [30]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.initializers import Constant

# define model
model = Sequential()
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_length,
                            trainable=False)

model.add(embedding_layer)
model.add(Conv1D(filters=600, kernel_size=1, activation='tanh'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(5, activation='softmax'))
print(model.summary())

# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the model
model.fit(X_train_pad, y_train, batch_size=128, epochs=50, validation_data=(X_test_pad, y_test), verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 199, 300)          14390400  
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 199, 600)          180600    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 99, 600)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 59400)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 297005    
Total params: 14,868,005
Trainable params: 477,605
Non-trainable params: 14,390,400
_________________________________________________________________
None
Train on 18000 samples, validate on 4500 samples
Epoch 1/50
 - 3s - loss: 1.5952 - acc: 0.2211 - val_loss: 1.5856 - val